In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import codecs
import re
from webdriver_manager.chrome import ChromeDriverManager
from io import StringIO
from bs4 import BeautifulSoup
import base64
from PIL import Image
from io import BytesIO
import contextlib
from selenium.common.exceptions import InvalidArgumentException
import time
import pandas as pd
import math
pd.set_option('display.max_rows', 1000)


# ignore warnings
pd.options.mode.chained_assignment = None

In [4]:
def start_driver():
    service = Service(executable_path=r"C:\chromedriver.exe")
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [73]:
driver = start_driver()

# Log In

In [75]:
# go to grailed login page
driver.get('https://www.grailed.com/')

# click login button
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@data-testid="login-btn"]'))).click()

# click login with email
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@data-cy="login-with-email"]'))).click()

In [76]:
# define username and password
username='username'
password='password'

# enter username and password
driver.find_element(By.XPATH, '//*[@id="email"]').send_keys(username)
driver.find_element(By.XPATH, '//*[@id="password"]').send_keys(password)
# driver.find_element(By.XPATH, '//*[@id="password"]').send_keys(Keys.RETURN)

In [79]:
# click log in button
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@data-cy="auth-login-submit"]'))).click()

# Get Current Products

In [181]:
# go to active listings
driver.get('https://www.grailed.com/sell/for-sale')

In [183]:
# scroll down to bottom while allowing time for products to load
curr_scrollheight = driver.execute_script("return document.documentElement.scrollHeight")
for i in range(0,100):
    page = driver.find_element("tag name","html")
    page.send_keys(Keys.END)
    time.sleep(2)
    new_scrollheight = driver.execute_script("return document.documentElement.scrollHeight")
    if new_scrollheight != curr_scrollheight:
        curr_scrollheight = new_scrollheight
    else:
        break

In [185]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [186]:
# get list of products on page
from bs4 import BeautifulSoup
import re

# empty vectors for variable storage
links = []
product_names = []
prices = []
og_prices = []
dates = []
        
# update page source
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

# get soups
product_soup = soup.find_all("p",attrs={"data-cy":"listing-title"})
price_soup = soup.find_all("div",attrs={"class":"Price_root__l_ELx"})
date_soup = soup.find_all("span",attrs={"class":"ListingAge_dateAgo__JEsJV"})

# get number of sold items, which are stored at the end of the page
num_sold_items = len(soup.find_all(attrs={"class":"icon tile__inventory-tag sold-tag"}))

# get product names and links
for tag in product_soup:
    p_name = re.split('title">|</p>', str(tag))[-2]
    product_names.append(p_name)

# get prices
for tag in price_soup:
    tag_prices = tag.find_all('p')
    # products whose price hasnt been dropped
    if len(tag_prices) == 1:
        prices.append(int(re.split('\\$', tag_prices[0].text)[-1]))
        og_prices.append(int(re.split('\\$', tag_prices[0].text)[-1]))
    # products w/ more than 1 price in their tag, indicating price has been dropped
    else:
        prices.append(int(re.split('\\$', tag_prices[0].text)[-1]))
        og_prices.append(int(re.split('\\$', tag_prices[1].text)[-1]))

# get product dates
for tag in date_soup:
    if "strikeThrough" in str(tag):
        date = re.split('">|<span', str(tag))[-3].replace(u'\xa0', u' ')
        dates.append(date)
    else:
        date = re.split('">|</span>', str(tag))[-2].replace(u'\xa0', u' ')
        dates.append(date)

# quantify dates from date strings
dates_numeric = []
for date in dates:
    if 'minutes' in date:
        date_numeric = 0
        dates_numeric.append(date_numeric)
    elif 'hour' in date:
        date_numeric = 0
        dates_numeric.append(date_numeric)
    elif 'days' in date:
        date_numeric = int(re.sub('[^0-9]','', date))
        dates_numeric.append(date_numeric)
    elif 'day' in date:
        date_numeric = int(re.sub('[^0-9]','', date))
        dates_numeric.append(date_numeric)
    elif 'month' in date:
        date_numeric = 30
        dates_numeric.append(date_numeric)
    elif 'minute' in date:
        date_numeric = 0
        dates_numeric.append(date_numeric)
        
#     print(date, date_numeric, len(dates_numeric))    

# get link
for a in soup.find_all('a', href=True):
    if 'listings' in a['href']:
        if 'edit' not in a['href']:
            link = "https://grailed.com/" + a['href']
            links.append(link)

In [189]:
for x in [product_names, prices, dates, dates_numeric, links]:
    print(len(x))

391
391
391
391
391


In [191]:
# dataframe w/ prices
dict = {'Item': product_names, 'Price': prices, 'OG Price': og_prices, 'Last Updated': dates, 'last_update_numeric': dates_numeric, 'Link': links} 
# dict = {'Item': product_names, 'Link': links, 'Price': prices} 

df = pd.DataFrame(dict)
display(df)

,Item,Price,OG Price,Last Updated,last_update_numeric,Link
0,Rob Thomas Chip Tooth Tour 2019 T Shirt Size X...,18,20,4 minutes ago,0,https://grailed.com//listings/69159734-canvas-...
1,Staples Crewneck Sweatshirt Size L 90s Office ...,22,25,4 minutes ago,0,https://grailed.com//listings/69159888-gildan-...
2,SOJA Reggae Band T Shirt Size S American Appar...,18,20,4 minutes ago,0,https://grailed.com//listings/69159757-america...
3,Apple Park Infinity Loop T Shirt Size XXS Blac...,18,20,4 minutes ago,0,https://grailed.com//listings/69159832-apple-a...
4,Spongebob Squarepants Tee Sz M Pink Is The Col...,14,16,4 minutes ago,0,https://grailed.com//listings/69159915-divided...
5,90s Maxtor Hard Drives Promo T Shirt Sz XL Dri...,18,20,4 minutes ago,0,https://grailed.com//listings/69159931-gildan-...
6,Trace Adkins X Tour 2010 T Shirt Size XL Doubl...,13,15,5 minutes ago,0,https://grailed.com//listings/69159948-gildan-...
7,NFL New England Patriots Zubaz Hoodie Size L N...,36,40,5 minutes ago,0,https://grailed.com//listings/69159975-nfl-nfl...
8,20th Century Fox Logo History T Shirt Size XL ...,14,16,5 minutes ago,0,https://grailed.com//listings/69159997-hanes-2...
9,Hello Kitty T Shirt Size S Kawaii Drink Jar Wh...,10,12,5 minutes ago,0,https://grailed.com//listings/69160014-fruit-o...


# Drop Price By a Percentage

In [194]:
# SET DISCOUNT AND FLOOR PRICE
percent_discount = 10
floor_price = 10

In [196]:
# get items that haven't been updated in at least a week
pricedrop_df = df.loc[(df['last_update_numeric'] >= 7) & (df['Price'] > floor_price)].reset_index(drop=True).drop_duplicates()

In [198]:
pricedrop_df

,Item,Price,OG Price,Last Updated,last_update_numeric,Link
0,Boston Calling Music Festival T Shirt Size XL ...,14,14,about 1 month ago,30,https://grailed.com//listings/69159190-gildan-...
1,Barbie Golfer T Shirt Size L Golf Pink Unisex ...,16,16,about 1 month ago,30,https://grailed.com//listings/69159176-hanes-b...
2,90s Reebok Windbreaker Jacket Sz L Blue Teal W...,23,23,about 1 month ago,30,https://grailed.com//listings/69159157-reebok-...
3,VTG Bushmills Red Bush Irish Whiskey Hat Baseb...,13,13,about 1 month ago,30,https://grailed.com//listings/69159117-otto-vt...
4,Jim Beam Basketball Shirt Size XL Throw Down T...,18,18,about 1 month ago,30,https://grailed.com//listings/69159085-hanes-j...
5,Uniqlo UT UItraman Zero T Shirt Size S Ultraseven,13,13,about 1 month ago,30,https://grailed.com//listings/69158874-uniqlo-...
6,DC Shoes Black Red Plaid Flexfit Hat Size 6.87...,13,13,about 1 month ago,30,https://grailed.com//listings/69158862-dc-dc-s...
7,NILS Resort Collection Fleece Jacket Sz L Full...,40,40,about 1 month ago,30,https://grailed.com//listings/69158689-gildan-...
8,Taco Bell Expect Mas Fire Sauce T Shirt Size L...,17,17,about 1 month ago,30,https://grailed.com//listings/69107077-canvas-...
9,Nike Venture Runner Sneakers Mens Size 13 Ratt...,40,40,about 2 months ago,30,https://grailed.com//listings/69052254-nike-ni...


In [200]:
print('Number of items to be discounted:', str(len(pricedrop_df)))

Number of items to be discounted: 272


In [202]:
# price drop loop
for i in range(0, len(pricedrop_df)):
    item = pricedrop_df.iloc[i,list(pricedrop_df.columns).index('Item')]
    link = pricedrop_df.iloc[i,list(pricedrop_df.columns).index('Link')]
    curr_price = pricedrop_df.iloc[i,list(pricedrop_df.columns).index('Price')]
    reduced_price = curr_price - (curr_price * (percent_discount*0.01))
    reduced_price = int(round(reduced_price, 0))
    
    # fix rounding issues
    if percent_discount == 10 and reduced_price/curr_price > 0.9:
        reduced_price = int(math.floor(0.9* curr_price))
        
    # if reduced_price < floor_price:
    #     reduced_price = floor_price
        
    # don't waste time on products already at the floor price / don't attempt to drop prices for products whose reduced_price is greater than the current price    
    if reduced_price >= curr_price:
        print('Item '+ str(i+1) + ' of ' + str(len(pricedrop_df)), '('+item+')\n', 'Old Price: '+ str(curr_price), 'New Price: '+ str(reduced_price))
        print('Item already at or below floor price.')
        
    else:
        print('Item '+ str(i+1) + ' of ' + str(len(pricedrop_df)), '('+item+')\n', 'Old Price: '+ str(curr_price), 'New Price: '+ str(reduced_price))
        driver.get(link)
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 600)")
        time.sleep(1)
    #     try:
        price_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//button[text()="Price Drop By"]')))
        price_element.click()
        time.sleep(2)
        price_input = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//input[@placeholder='Set a new price for your item']")))
        price_input.send_keys(str(reduced_price))
        pricedrop_button = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//button[@class='Button_button__30ukX Button_large__r_Cnd Button_important__eJRlB SubmitButton_button___UVm_']")))
        time.sleep(1)
        pricedrop_button.click()
        time.sleep(1)
#     except:
#         print('Check authenticity for', item)
#         pass
    
print('Done.')
driver.quit()

Item 1 of 272 (Boston Calling Music Festival T Shirt Size XL Skull Graphic)
 Old Price: 14 New Price: 12
Item 2 of 272 (Barbie Golfer T Shirt Size L Golf Pink Unisex Graphic Tee)
 Old Price: 16 New Price: 14
Item 3 of 272 (90s Reebok Windbreaker Jacket Sz L Blue Teal White Full Zip)
 Old Price: 23 New Price: 20
Item 4 of 272 (VTG Bushmills Red Bush Irish Whiskey Hat Baseball Cap)
 Old Price: 13 New Price: 11
Item 5 of 272 (Jim Beam Basketball Shirt Size XL Throw Down The Rock Promo)
 Old Price: 18 New Price: 16
Item 6 of 272 (Uniqlo UT UItraman Zero T Shirt Size S Ultraseven)
 Old Price: 13 New Price: 11
Item 7 of 272 (DC Shoes Black Red Plaid Flexfit Hat Size 6.875-7.25)
 Old Price: 13 New Price: 11
Item 8 of 272 (NILS Resort Collection Fleece Jacket Sz L Full Zip Hood 80s)
 Old Price: 40 New Price: 36
Item 9 of 272 (Taco Bell Expect Mas Fire Sauce T Shirt Size L 911 Level 10)
 Old Price: 17 New Price: 15
Item 10 of 272 (Nike Venture Runner Sneakers Mens Size 13 Rattan Orange)
 Old Pr

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7CD732775+28773]
	(No symbol) [0x00007FF7CD69AFB0]
	(No symbol) [0x00007FF7CD53552A]
	(No symbol) [0x00007FF7CD588EAE]
	(No symbol) [0x00007FF7CD58919C]
	(No symbol) [0x00007FF7CD5D23F7]
	(No symbol) [0x00007FF7CD5AEFDF]
	(No symbol) [0x00007FF7CD5CF1A4]
	(No symbol) [0x00007FF7CD5AED43]
	(No symbol) [0x00007FF7CD57A548]
	(No symbol) [0x00007FF7CD57B6B1]
	GetHandleVerifier [0x00007FF7CDA5F45D+3358029]
	GetHandleVerifier [0x00007FF7CDA7430D+3443709]
	GetHandleVerifier [0x00007FF7CDA683FD+3394797]
	GetHandleVerifier [0x00007FF7CD7F929B+842635]
	(No symbol) [0x00007FF7CD6A654F]
	(No symbol) [0x00007FF7CD6A1FA4]
	(No symbol) [0x00007FF7CD6A213D]
	(No symbol) [0x00007FF7CD691629]
	BaseThreadInitThunk [0x00007FFB3F287374+20]
	RtlUserThreadStart [0x00007FFB3F5DCC91+33]
